In [1]:
import os

In [2]:
%pwd

'/mnt/d/Me/Ai/NLP/Krish Naik/end-to-end-ds/datascienceproject/datascienceproject/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/mnt/d/Me/Ai/NLP/Krish Naik/end-to-end-ds/datascienceproject/datascienceproject'

In [12]:
from pathlib import Path
from dataclasses import dataclass

@dataclass
class ModelTrainerConfiguration:
    root_dir: Path
    train_data: Path
    test_data: Path
    model_name: Path
    alpha: float
    l1_ratio: float
    target_columns: str

In [13]:
from src.datascienceproject.constants import *
from src.datascienceproject.utils.common import read_yaml, create_directory

In [14]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.schema = read_yaml(schema_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directory([self.config.artifacts_root])
        
    def get_model_training_config(self) -> ModelTrainerConfiguration:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMNS
        create_directory([config.root_dir])
        
        model_trainer_config = ModelTrainerConfiguration(
            root_dir = config.root_dir,
            train_data = config.train_data,
            test_data = config.test_data,
            model_name = config.model_name,
            alpha = params.alpha,
            l1_ratio = params.l1_ratio,
            target_columns = schema.name,
        )
        return model_trainer_config

In [15]:
import os
from sklearn.linear_model import ElasticNet
from src.datascienceproject import logger
import joblib
import pandas as pd

In [16]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfiguration):
        self.config = config
        
    def train(self):
        
        train_data = pd.read_csv(self.config.train_data)
        test_data = pd.read_csv(self.config.test_data)
        
        train_X = train_data.drop([self.config.target_columns], axis=1)
        train_y = train_data[[self.config.target_columns]]
        
        test_X = test_data.drop([self.config.target_columns], axis=1)
        test_y = test_data[[self.config.target_columns]]
        
        lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio)
        lr.fit(train_X, train_y)
        
        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))

In [17]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_model_training_config()
    data_validation = ModelTrainer(data_validation_config)
    data_validation.train()
except Exception as e:
    raise e

[2024-12-29 20:03:59,042: INFO: common: yaml file -> config/config.yaml loadded successfuly]
[2024-12-29 20:03:59,465: INFO: common: yaml file -> schema.yaml loadded successfuly]
[2024-12-29 20:03:59,539: INFO: common: yaml file -> params.yaml loadded successfuly]
[2024-12-29 20:03:59,747: INFO: common: created directory at artificts]
[2024-12-29 20:03:59,751: INFO: common: created directory at artificts/model_trainer]
